In [4]:
import brightway2 as bw
import numpy as np
import pandas as pd
from copy import copy

# Setup the project

### TODO -> choose project

In [5]:
project = 'GSA for ecoinvent'

In [6]:
if project not in bw.projects:
    bw.projects.create_project(project)
bw.projects.set_current(project)

# Import older versions of databases

### TODO -> give paths to databases files
for now let's use ecoinvent 3.3 cutoff and EXIOBASE 2.2

In [7]:
# Older databases
ei33_path = '/Users/akim/Documents/LCA files/ecoinvent 3.3 cutoff/datasets'  
ex22_path = '/Users/akim/Documents/LCA files/exiobase 2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
ag12_path = '/Users/akim/Documents/LCA files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'

# Newer databases
ei36_path = '/Users/akim/Documents/LCA files/ecoinvent 3.6 cutoff/datasets'  
# ex34_path = '/Users/akim/Documents/LCA files/exiobase 3.4/IOT_2011_pxp/' #TODO
ag13_path = '/Users/akim/Documents/LCA files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'

In [8]:
bw.databases

Databases dictionary with 8 object(s):
	Agribalyse 1.2 - ecoinvent 3.3 cutoff
	Agribalyse 1.2 - ecoinvent 3.3 cutoff - new biosphere
	Agribalyse 1.3 - ecoinvent 3.6 cutoff
	Agribalyse 1.3 - ecoinvent 3.6 cutoff - new biosphere
	EXIOBASE 2.2
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.6 cutoff

## 1. Biosphere3

In [9]:
bw.bw2setup()

Biosphere database already present!!! No setup is needed


## 2. Ecoinvent 3.3

In [10]:
ei33_name = "ecoinvent 3.3 cutoff"  
if ei33_name in bw.databases:
    print(ei33_name + " database already present!!! No import is needed")
else:
    ei33 = bw.SingleOutputEcospold2Importer(ei33_path, ei33_name)
    ei33.apply_strategies()
    ei33.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei33.statistics()

ecoinvent 3.3 cutoff database already present!!! No import is needed


In [11]:
if ei33_name in bw.databases:
    print(ei33_name + " database already present!!! No import is needed")
else:
    ei33.write_database()

ecoinvent 3.3 cutoff database already present!!! No import is needed


## 3. Exiobase 2.2

In [12]:
# 2. Exiobase 
from bw2io.importers.exiobase import Exiobase22Importer

In [13]:
ex22_name = 'EXIOBASE 2.2'
if ex22_name in bw.databases:
    print(ex22_name + " database already present!!! No import is needed")
else:
    ex22 = Exiobase22Importer(ex22_path, ex22_name)
    ex22.apply_strategies()

EXIOBASE 2.2 database already present!!! No import is needed


In [14]:
if ex22_name in bw.databases:
    print(ex22_name + " database already present!!! No import is needed")
else:
    ex22.write_database()

EXIOBASE 2.2 database already present!!! No import is needed


## 4. Agribalyse 1.2
better see: [how to import](https://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20Importing%20Agribalyse%20with%20Ecoinvent%202.2.ipynb)

In [15]:
def import_agribalyse12(ag12_path, ag12_name, ei_name):

    ag12_ei_name = ag12_name + ' - ' + ei_name
    
    if ei_name not in bw.databases:
        print('Cannot find database: ' + ei_name)
        return
    
    if ag12_ei_name in bw.databases:
        print(ag12_ei_name + " database already present!!! No import is needed")
    else:
        ag12_ei = bw.SingleOutputEcospold1Importer(ag12_path, ag12_ei_name)
        ag12_ei.apply_strategies()
        ag12_ei.statistics()
        
        ag12_ei.write_excel(True)
        from bw2io.strategies.simapro import normalize_simapro_biosphere_categories, normalize_simapro_biosphere_names
        ag12_ei.apply_strategy(normalize_simapro_biosphere_categories)
        ag12_ei.apply_strategy(normalize_simapro_biosphere_names)

        from bw2io.strategies import link_iterable_by_fields
        import functools
        ag12_ei.apply_strategy(functools.partial(link_iterable_by_fields, other=bw.Database("biosphere3"), kind="biosphere"))
        ag12_ei.statistics()

        ag12_ei.write_excel(True)
        
        ag12_ei_new_biosphere_name = ag12_ei_name + " - new biosphere"
        bw.Database(ag12_ei_new_biosphere_name).register()
        ag12_ei.add_unlinked_flows_to_biosphere_database(ag12_ei_new_biosphere_name)

        def get_unlinked(data):
            for ds in ag12_ei.data:
                for exc in ds['exchanges']:
                    if exc['type'] == 'production' and exc['name'] == 'Alfalfa, conventional, for animal feeding, at farm gate':
                        return ds, exc

        ds, exc = get_unlinked(ag12_ei.data)

        from bw2io.strategies import link_technosphere_based_on_name_unit_location
        ag12_ei.apply_strategy(link_technosphere_based_on_name_unit_location)
        ag12_ei.statistics()

        def get_unlinked(data):
            for ds in ag12_ei.data:
                for exc in ds['exchanges']:
                    if exc['type'] == 'production' and not exc.get('input'):
                        return ds, exc

        ds, exc = get_unlinked(ag12_ei.data)

        for ds in ag12_ei.data:
            for exc in ds['exchanges']:
                if exc['type'] == 'production' and not exc.get('input'):
                    print("Fixing:", exc['name'])
                    exc['type'] = 'technosphere'

        ag12_ei.apply_strategy(link_technosphere_based_on_name_unit_location)
        ei_name_short = ei_name #TODO
        ag12_ei.apply_strategy(functools.partial(link_technosphere_based_on_name_unit_location, external_db_name=ei_name_short))
        ag12_ei.statistics()

        ag12_ei.add_unlinked_activities()
        ag12_ei.statistics()

        for ds in ag12_ei.data:
            del ds['code']

        from bw2io.strategies import set_code_by_activity_hash
        ag12_ei.apply_strategy(set_code_by_activity_hash)

        ag12_ei.apply_strategy(functools.partial(
                link_iterable_by_fields,
                other=ag12_ei.data,
                fields=('name', 'location', 'unit'),
                relink=True
        ))
        ag12_ei.statistics()
        
        if ag12_ei_name in bw.databases:
            print(ag12_ei_name + " database already present!!! No import is needed")
        else:
            ag12_ei.write_database()

In [16]:
ag12_name = 'Agribalyse 1.2'
import_agribalyse12(ag12_path, ag12_name, ei33_name)

Agribalyse 1.2 - ecoinvent 3.3 cutoff database already present!!! No import is needed


In [17]:
bw.databases

Databases dictionary with 8 object(s):
	Agribalyse 1.2 - ecoinvent 3.3 cutoff
	Agribalyse 1.2 - ecoinvent 3.3 cutoff - new biosphere
	Agribalyse 1.3 - ecoinvent 3.6 cutoff
	Agribalyse 1.3 - ecoinvent 3.6 cutoff - new biosphere
	EXIOBASE 2.2
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.6 cutoff

## 5. HEIA

### TODO -> ask Andi for files

# Import newer versions of databases

## 1. Ecoinvent 3.6 cutoff

In [18]:
ei36_name = "ecoinvent 3.6 cutoff"
if ei36_name in bw.databases:
    print(ei36_name + " database already present!!! No import is needed")
else:
    ei36 = bw.SingleOutputEcospold2Importer(ei36_path, ei36_name)
    ei36.apply_strategies()
    ei36.match_database(db_name='biosphere3',fields=('name', 'category', 'unit', 'location'))
    ei36.statistics()

ecoinvent 3.6 cutoff database already present!!! No import is needed


In [19]:
if ei36_name in bw.databases:
    print(ei36_name + " database already present!!! No import is needed")
else:
    ei36.write_database()

ecoinvent 3.6 cutoff database already present!!! No import is needed


In [23]:
bw.databases

Databases dictionary with 7 object(s):
	Agribalyse 1.2 - ecoinvent 3.3 cutoff
	Agribalyse 1.2 - ecoinvent 3.3 cutoff - new biosphere
	Agribalyse 1.3 - ecoinvent 3.6 cutoff - new biosphere
	EXIOBASE 2.2
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.6 cutoff

## 2. Agribalyse 1.3

In [22]:
# del bw.databases['Agribalyse 1.3 - ecoinvent 3.6 cutoff']

Vacuuming database 


### TODO -> Chris, Rhythima, check if migrations are correct

In [24]:
ei_name = ei36_name

In [25]:
ag13_name = 'Agribalyse 1.3'
ag13_ei_name = ag13_name + ' - ' + ei_name
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    ag13_ei = bw.SimaProCSVImporter(ag13_path, ag13_ei_name)
    ag13_ei.apply_strategies()
    
    # Apply all migrations with previous versions of ecoinvent
    ag13_ei.migrate('simapro-ecoinvent-3.3')
    
    # Update US locations
    from bw2io.strategies.locations import update_ecoinvent_locations
    ag13_ei = update_ecoinvent_locations(ag13_ei)
    
    # Biosphere flows
    ag13_ei_new_biosphere_name = ag13_ei_name + " - new biosphere"
    bw.Database(ag13_ei_new_biosphere_name).register()
    ag13_ei.add_unlinked_flows_to_biosphere_database(ag13_ei_new_biosphere_name)
    
    # Add unlinked waste flows as new activities
    ag13_ei.add_unlinked_activities()
    
    ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))
    ag13_ei.statistics()

Extracted 1189 unallocated datasets in 3.47 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: assign_only_product_as_production
Applying strategy: drop_unspecified_subcategories
Applying strategy: sp_allocate_products
Applying strategy: fix_zero_allocation_products
Applying strategy: split_simapro_name_geo
Applying strategy: strip_biosphere_exc_locations
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: set_code_by_activity_hash
Applying strategy: link_technosphere_based_on_name_unit_location
Applying strategy: change_electricity_unit_mj_to_kwh
Applying strategy: set_lognormal_loc_value_uncertainty_safe
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_simapro_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: normalize_simapro_biosphere_names
Applying strategy: migrate_exchanges
Applying strategy: fix_localized_water_flows
A

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 10/25/2019 12:17:15
  Finished: 10/25/2019 12:17:15
  Total time elapsed: 00:00:00
  CPU %: 107.00
  Memory %: 2.26
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
1700 datasets
115259 exchanges
528 unlinked exchanges
  Type technosphere: 35 unique unlinked exchanges


### Define some of the migrations manually.
- Most of them are minor changes in names of activities and reference products
- Some activities contain `multiplier` field if the unit conversion is needed or reference products are not identical

In [26]:
agribalyse13_ecoinvent36_names = {
    'fields': ['name'],
    'data': [
        (
            ['extrusion production, plastic film'],
            {
                'name': 'extrusion, plastic film'
            }
        ),
        (
            ['extrusion production, plastic pipes'],
            {
                'name': 'extrusion, plastic pipes'
            }
        ),
        (
            ['blow moulding production'],
            {
                'name': 'blow moulding'
            }
        ),
        (
            ['calendering production, rigid sheets'],
            {
                'name': 'calendering, rigid sheets'
            }
        ),
        (
            ['steam production in chemical industry'],
            {
                'name': 'steam production, in chemical industry',
                'reference product': 'steam, in chemical industry',
                'unit': 'kilogram',
                'multiplier': 1/2.75, # change of units, see comment on this activity in ecoinvent
            }
        ),
        (
            ['water production, completely softened, from decarbonised water, at user'],
            {
                'name': 'water production, completely softened',
                'reference product': 'water, completely softened'
            }
        ),
        (
            ['market for water, decarbonised, at user'],
            {
                'name': 'market for water, decarbonised',
                'reference product': 'water, decarbonised'
            }
        ),
        (
            ['textile production, jute'],
            {
                'name': 'textile production, jute, weaving'
            }
        ),
        (
            ['market for transport, freight, sea, transoceanic ship'],
            {
                'name': 'market for transport, freight, sea, container ship with reefer, cooling',
                'reference product': 'transport, freight, sea, container ship with reefer, cooling'
            }
        ),
        (
            ['transport, freight, sea, transoceanic ship'],
            {
                'name': 'transport, freight, sea, container ship with reefer, cooling',
                'reference product': 'transport, freight, sea, container ship with reefer, cooling'
            }
        ),
        (
            ['oxidation of manganese dioxide'],
            {
                'reference product': 'potassium permanganate',
                'multiplier': 90/1000 # change of reference product, see comment on this activity in ecoinvent
            }
        ),
        (
            ['irrigation'],
            {
                'name': 'market for irrigation',
            }
        ),
    ]
}

bw.Migration("agribalyse13-ecoinvent36-names").write(
    agribalyse13_ecoinvent36_names,
    description="Change names of some activities"
)

### Define migrations for market groups

In [27]:
agribalyse13_ecoinvent36_market_groups = {
    'fields': ['name', 'location'],
    'data': [
        (
            ['market for electricity, low voltage', 'BR'],
            {
                'name': 'market group for electricity, low voltage'
            }
        ),
        (
            ['market for electricity, medium voltage', 'BR'],
            {
                'name': 'market group for electricity, medium voltage'
            }
        ),
    ]
}

bw.Migration("agribalyse13-ecoinvent36-market-groups").write(
    agribalyse13_ecoinvent36_market_groups,
    description="Change markets to market groups for Brazil"
)

In [28]:
ag13_ei.migrate('agribalyse13-ecoinvent36-names')
ag13_ei.migrate('agribalyse13-ecoinvent36-market-groups')
ag13_ei.match_database(ei_name, fields=('reference product','location', 'unit', 'name'))
ag13_ei.statistics()

Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: migrate_datasets
Applying strategy: migrate_exchanges
Applying strategy: link_iterable_by_fields
1700 datasets
115259 exchanges
334 unlinked exchanges
  Type technosphere: 21 unique unlinked exchanges


(1700, 115259, 334)

### Allocate by production volume

In [29]:
from utils import *

In [30]:
ei36 = bw.Database(ei_name)

In [31]:
# Create mapping between unlinked exchanges and ecoinvent activities that constitute each unlinked exchange.
# In this case we do not need to do it manually since all unlinked exchanges need to be allocated geographically.
# Example: (market for lime, GLO) is split by production volume into (market for lime, RoW) & (market for lime, RER).
# Mapping is a list of dictionaries where each dictionary corresponds to an unlinked exchange.
# The key is the name of the unlinked exchange and the values are ecoinvent activities codes.

def create_location_mapping(ag13_ei, ei_name):
    
    ei36 = bw.Database(ei_name)
    
    unlinked_list = list(ag13_ei.unlinked)
    len_unlinked  = len(unlinked_list)

    mapping = [0]*len_unlinked

    for u in range(len_unlinked):
        new_el = {}
        name = unlinked_list[u]['name'] 
        loc  = unlinked_list[u]['location']
        acts_codes = [act['code'] for act in ei36 if name == act['name']]
        new_el[ (name, loc) ] = acts_codes
        mapping[u] = new_el
        
    return mapping

In [32]:
mapping = create_location_mapping(ag13_ei, ei_name)

In [33]:
agg = modify_exchanges(ag13_ei, mapping, ei_name)

Applying strategy: link_iterable_by_fields


In [34]:
agg.statistics()

1700 datasets
115943 exchanges
0 unlinked exchanges
  


(1700, 115943, 0)

### Write database

In [35]:
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    agg.write_database()

NonuniqueCode: The following activities have non-unique codes: ['ammonium nitrate phosphate production', 'diammonium phosphate production', 'monoammonium phosphate production']

In [36]:
a = 'ammonium nitrate phosphate production'
b = 'diammonium phosphate production'
c = 'monoammonium phosphate production'

In [37]:
for act in agg.data:
    if act['name'] in [a,b,c]:
        print(act)

{'categories': ('Materials/fuels',), 'name': 'ammonium nitrate phosphate production', 'unit': 'kilogram', 'comment': '(1,1,5,1,1,na); assimilation MAP', 'location': 'RER', 'type': 'process', 'code': 'de221991cc69f37976042f05f448c94c', 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'}
{'categories': ('Materials/fuels',), 'name': 'ammonium nitrate phosphate production', 'unit': 'kilogram', 'comment': '(1,1,5,1,1,na); assimilation MAP', 'location': 'RER', 'type': 'process', 'code': 'de221991cc69f37976042f05f448c94c', 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'}
{'categories': ('Materials/fuels',), 'name': 'diammonium phosphate production', 'unit': 'kilogram', 'comment': "Mineral fertilizers. Model of transport: 'MT MAP DAP', weight transported in tons = 5,9E-02. Pedigree-Matrix='(3,3,2,1,2,na)'.", 'location': 'RER', 'type': 'process', 'code': '64037e162f6d6d1048470c3a1135f4fb', 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'}
{'categories': ('Materials/fuels',), 'name': 'di

In [38]:
# Remove repetitive activities
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'ammonium nitrate phosphate production', 
                 'unit': 'kilogram', 
                 'comment': '(1,1,5,1,1,na); assimilation MAP', 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': 'de221991cc69f37976042f05f448c94c', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'diammonium phosphate production', 
                 'unit': 'kilogram', 
                 'comment': "Mineral fertilizers. Model of transport: 'MT MAP DAP', weight transported in tons = 5,9E-02. Pedigree-Matrix='(3,3,2,1,2,na)'.", 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': '64037e162f6d6d1048470c3a1135f4fb', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})
agg.data.remove({'categories': ('Materials/fuels',), 
                 'name': 'monoammonium phosphate production', 
                 'unit': 'kilogram', 
                 'comment': '', 
                 'location': 'RER', 
                 'type': 'process', 
                 'code': '6d61eb45c1d285073770aa839426d97c', 
                 'database': 'Agribalyse 1.3 - ecoinvent 3.6 cutoff'})


In [39]:
if ag13_ei_name in bw.databases:
    print(ag13_ei_name + " database already present!!! No import is needed")
else:
    agg.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Title: Writing activities to SQLite3 database:
  Started: 10/25/2019 12:18:14
  Finished: 10/25/2019 12:18:22
  Total time elapsed: 00:00:07
  CPU %: 96.90
  Memory %: 3.32
Created database: Agribalyse 1.3 - ecoinvent 3.6 cutoff


# Exiobase 3.4.7